# **Evidencia 1: Generación del set de datos** 
## Equipo:
* Rubén Sánchez Mayén A01378379
* Octavio Andrick Sánchez Perusquia A01378649
* Joan Daniel Guerrero García A01378052

In [9]:
!unzip evidencia1
!unzip evidencia1/documentos-genuinos
!unzip evidencia1/'documentos-con texto de otros'
!unzip evidnecia1/documentos-sospechosos

Archive:  documentos-sospechosos.zip
  inflating: docmentos-sospechosos/FID-14.txt  
  inflating: docmentos-sospechosos/FID-05.txt  
  inflating: docmentos-sospechosos/FID-08.txt  
  inflating: docmentos-sospechosos/FID-02.txt  
  inflating: docmentos-sospechosos/FID-12.txt  
  inflating: docmentos-sospechosos/FID-01.txt  
  inflating: docmentos-sospechosos/FID-15.txt  
  inflating: docmentos-sospechosos/FID-10.txt  
  inflating: docmentos-sospechosos/FID-11.txt  
  inflating: docmentos-sospechosos/FID-07.txt  
  inflating: docmentos-sospechosos/FID-13.txt  
  inflating: docmentos-sospechosos/FID-09.txt  
  inflating: docmentos-sospechosos/FID-06.txt  
  inflating: docmentos-sospechosos/FID-03.txt  
  inflating: docmentos-sospechosos/FID-04.txt  


In [2]:
# Función para importar un archivo de texto
def importar_texto(nombre_archivo):
  with open(nombre_archivo) as archivo:
      return archivo.read()

In [13]:
#Importar datos para entrenamiento y pruebas
import os
dir_genuinos = '/content/documentos-genuinos'
dir_reutilizados = '/content/documentos-con texto de otros'
dir_sospechosos = '/content/docmentos-sospechosos'

def texto_de_archivos(directorio):
  lista_textos = []
  for subdir, dirs, files in os.walk(directorio):
      for file in files:
          lista_textos.append(importar_texto(f'{directorio}/{file}'))
  return lista_textos

#Colocar textos en listas
textos_sospechosos = texto_de_archivos(dir_sospechosos)
textos_reutilizados = []
textos_genuinos = []
#Separar textos reutilizados y genuinos de la carpeta de sospechosos
for i in range(len(textos_sospechosos)):
  if i % 2 == 0:
    textos_reutilizados.append(textos_sospechosos[i])
  else:
    textos_genuinos.append(textos_sospechosos[i])

#Cargar el resto de textos según su originalidad
textos_reutilizados += texto_de_archivos(dir_reutilizados)
textos_genuinos += texto_de_archivos(dir_genuinos)

#Algunos ejemplos
print(textos_genuinos[0])
print(textos_reutilizados[0])

Machine learning algorithms are often vulnerable to adversarial examples that have imperceptible alterations from the original counterparts but can fool the state-of-the-art models. It is helpful to evaluate or even improve the robustness of these models by exposing the maliciously crafted adversarial examples. In this paper, we present TextFooler, a simple but strong baseline to generate adversarial text. By applying it to two fundamental natural language tasks, text classification and textual entailment, we successfully attacked three target models, including the powerful pre-trained BERT, and the widely used convolutional and recurrent neural networks. We demonstrate three advantages of this framework: (1) effective—it outperforms previous attacks by success rate and perturbation rate, (2) utility-preserving—it preserves semantic content, grammaticality, and correct types classified by humans, and (3) efficient—it generates adversarial text with computational complexity linear to th

In [88]:
#Generar dataset de entrenamiento
import distanciaCosenoTxts as disCos

#Función que a partir de una base de datos, calcula la distancia coseno
#para los enegramas especificados de cada texto en una lista objetivo, según la mayor 
#coincidencia en la base de datos
def obtenerDistanciaDatos(base_datos, lista_objetivo, enegramas):
  x = []
  for i in range(len(lista_objetivo)):
    x.append([0]*enegramas)
    texto_objetivo = lista_objetivo[i]
    for texto_base in base_datos:
        for j in range(enegramas):
          distancia = disCos.distanciaEntreTextos([texto_objetivo, texto_base], j+1)[0][1]
          if distancia >= .8: #Detectar outliers y textos identicos
            break
          x[i][j] = max(distancia, x[i][j]) #La lista se encoje con cada outlier eliminado
  return x

In [89]:
ENEGRAMAS = 3

#Agregar la distancia de todos los textos reutilizados al nuevo dataset
xR = obtenerDistanciaDatos(textos_genuinos, textos_reutilizados, ENEGRAMAS)
yR = [1]*len(xR)

#Algunos ejemplos
print(xR[:3])

[[0.4949399496798402, 0.3910371502940005, 0.3376872344663547], [0.2602896031476768, 0.049160514400834666, 0.011629676092591607], [0.3268552425669442, 0.05852683750748628, 0.026462806201248152]]


In [92]:
#Agregar la distancia de n textos geuninos al dataset,
#donde n es igual al numero de textos reutilizados
xG = obtenerDistanciaDatos(textos_genuinos, textos_genuinos[:len(xR)], ENEGRAMAS)
yG = [0]*len(xG)

#Algunos ejemplos
print(xG[:3])

[[0.22631157994440146, 0.05300454699132827, 0.019534959790148375], [0.4174686989207958, 0.09658905768325902, 0.018163480488618104], [0.2775554665954842, 0.06804138174397717, 0.028782192785089703]]


In [93]:
#Unificar y serializar dataset
x = xR + xG
y = yR + yG
with open(r'/content/dataset_reutilizacion_texto.csv', 'w') as archivo:
  for i in range(len(x)):
    for j in range(ENEGRAMAS):
      archivo.write("%s," % (x[i][j]))
    archivo.write("%s\n" % y[i])